https://nni.readthedocs.io/en/stable/tutorials/hpo_quickstart_pytorch/model.html

In [17]:
import nni
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


params = {
    'features': 512,
    'lr': 0.001,
    'momentum': 0,
}

In [18]:
optimized_params = nni.get_next_parameter()
params.update(optimized_params)
print(params)
print(type(params))

{'features': 512, 'lr': 0.001, 'momentum': 0}
<class 'dict'>


In [19]:
training_data = datasets.FashionMNIST(root="data", train=True, download=True, transform=ToTensor())
test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform=ToTensor())

batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [20]:
for train_batch in train_dataloader:
    x, y = train_batch
    print(x.shape)
    print(y.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


In [21]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, params['features']),
            nn.ReLU(),
            nn.Linear(params['features'], params['features']),
            nn.ReLU(),
            nn.Linear(params['features'], 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=params['lr'], momentum=params['momentum'])

Using cuda device


In [22]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    return correct

In [23]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    accuracy = test(test_dataloader, model, loss_fn)
    nni.report_intermediate_result(accuracy)
nni.report_final_result(accuracy)

Epoch 1
-------------------------------
[2023-04-08 06:30:54] Intermediate result: 0.5516  (Index 0)
Epoch 2
-------------------------------
[2023-04-08 06:30:56] Intermediate result: 0.5616  (Index 1)
Epoch 3
-------------------------------
[2023-04-08 06:30:58] Intermediate result: 0.6011  (Index 2)
Epoch 4
-------------------------------
[2023-04-08 06:31:00] Intermediate result: 0.6302  (Index 3)
Epoch 5
-------------------------------
[2023-04-08 06:31:02] Intermediate result: 0.6425  (Index 4)
[2023-04-08 06:31:02] Final result: 0.6425
